In [76]:
import json

with open("whisper-v3-aligned-to-sw_all.json") as inf:
    data = json.load(inf)

In [77]:
def clean_text(text):
    if text.endswith("."):
        text = text[:-1]
    if text.endswith("!"):
        text = text[:-1]
    if text.endswith("?"):
        text = text[:-1]
    text = text.replace(", ", " ")
    return text.lower().strip()

In [78]:
def ucfirst(text):
    if len(text) == 0:
        return text
    return text[0].upper() + text[1:]

In [79]:
for item in data:
    data[item]["orig_clean"] = data[item]["orig"].replace("_", " ")
    if not "res" in data[item]:
        print(item)
        continue
    data[item]["res_clean"] = clean_text(data[item]["res"])
    if data[item]["orig_clean"] == data[item]["res_clean"]:
        data[item]["match"] = "simple"
        data[item]["corrected"] = data[item]["res"]

5278
5279


In [119]:
from jiwer import wer

for item in sorted(data.keys()):
    item_id = int(item)
    if not "res" in data[item]:
        continue
    if data[item]["orig_clean"] == data[item]["res_clean"]:
        continue
    # data[item]["wer"] = wer(data[item]["orig_clean"], data[item]["res_clean"])
    if item_id >= 4155 and item_id < 5144:
        print(f"{item_id}", wer(data[str(item_id + 1)]["orig_clean"], data[item]["res_clean"]), "||", data[str(item_id + 1)]["orig_clean"], "||", data[item]["res_clean"])
    elif item_id >= 5144:
        print(f"{item_id}", wer(data[str(item_id + 2)]["orig_clean"], data[item]["res_clean"]), "||", data[str(item_id + 2)]["orig_clean"], "||", data[item]["res_clean"])


4155 0.0 || huset var nämligen stort alldeles för stort egentligen och uppfört i två våningar med ett tredje vindsplan och i ena hörnan sköt ett torn upp ett riktigt trätorn med runda fönster || huset var nämligen stort alldeles för stort egentligen och uppfört i två våningar med ett tredje vindsplan och i ena hörnan sköt ett torn upp ett riktigt trätorn med runda fönster
4156 0.03225806451612903 || det förblev förstås oklart vad det var för fiskeflottor som förväntades komma farande över åkrarna här i marielund men vad spelade det för roll om man bara hade virke och fantasi || det förblev förstås oklart vad det var för fiskeflottor som förväntades komma farande över åkrarna här i marielund. men vad spelade det för roll om man bara hade virke och fantasi
4157 0.0 || det fanns hur mycket trä som helst i saras barndom för pappa var rik som ett troll || det fanns hur mycket trä som helst i saras barndom för pappa var rik som ett troll
4158 0.043478260869565216 || hon mindes livet som en s

In [ ]:
supplied = {
    "0065": "Det finns visserligen ishockey och fotboll, men långt ned i seriesystemet.",
    "0064": "Nej, han tillhör inte deckargenrens tuffingar."
}

for manual in supplied:
    data[manual]["match"] = "supplied"
    data[manual]["corrected"] = supplied[manual]

for manual in ["1569"]:
    data[manual]["match"] = "norm"
    data[manual]["corrected"] = data[manual]["res"]

for manual in ["0165"]:
    data[manual]["match"] = "spelling"
    data[manual]["corrected"] = data[manual]["orig"] + "."

for manual in ["2195", "2074", "3087"]:
    data[manual]["match"] = "spelling"
    data[manual]["corrected"] = ucfirst(data[manual]["orig_clean"])

for manual in ["0803", "2306", "2298", "0237", "0011", "0001", "0042", "0045", "0246", "0242", "0220", "0217", "0063",
               "0070"]:
    data[manual]["match"] = "spelling"
    data[manual]["corrected"] = ucfirst(data[manual]["orig_clean"]) + "."

for manual in ["2582"]:
    data[manual]["match"] = "misspeak"
    data[manual]["corrected"] = ucfirst(data[manual]["orig_clean"]) + "."

for manual in ["2686"]:
    data[manual]["match"] = "whisper"
    data[manual]["corrected"] = data[manual]["res"]


In [ ]:
# 0246: trädkronor : träkronor
# 0220: blåbärssoppa : blåbärshoppa


In [56]:
import soundfile as sf
from IPython.display import Audio
import numpy as np

def read_with_soundfile(filename):
    return sf.read(filename, channels=2, samplerate=44100, endian="BIG",
                   dtype="int16", format="RAW", subtype="PCM_16", start=16)

def display_id(text):
    filename = f'/Users/joregan/Playing/nst_swedish_tts/sw_pcms/mf/sw_all_mf_01_{text}.pcm'
    data, sr = read_with_soundfile(filename)
    data = data[:, 1]
    data = data.astype(np.float32) / 32768.0
    return Audio(data, rate=44100)

In [ ]:
misaligned = {
    "4584": "4585",
    "4585": "4586",
    "4586": "4587",
    "4587": "4588",
}

In [111]:
display_id("4155")

In [99]:
pruned = {x: y for (x, y) in data.items() if not "match" in y}

In [100]:
import collections

spruned = collections.OrderedDict(sorted(pruned.items()))

In [110]:
import json

with open("/tmp/tocheck.json", "w") as outf:
    json.dump(spruned, outf, indent=2)

with open("/tmp/all.json", "w") as outf:
    json.dump(dict(sorted(data.items())), outf, indent=2)